# Module 3. Deployment on MMS(Multi Model Server)
---

본 모듈에서는 모델의 배포(deployment)를 수행합니다. 노트북 실행에는 약 15분 가량 소요되며, 핸즈온 실습 시에는 25분을 권장드립니다.

<br>

## 1. Inference script
---

아래 코드 셀은 `src` 디렉토리에 SageMaker 추론 스크립트인 `inference.py`를 저장합니다.<br>

이 스크립트는 SageMaker 상에서 MMS(Multi Model Server)를 쉽고 편하게 배포할 수 이는 high-level 툴킷인 SageMaker inference toolkit의 인터페이스를
사용하고 있으며, 여러분께서는 인터페이스에 정의된 핸들러(handler) 함수들만 구현하시면 됩니다.

#### MMS(Multi Model Server)란?
- [https://github.com/awslabs/multi-model-server](https://github.com/awslabs/multi-model-server) (2017년 12월 초 MXNet 1.0 릴리스 시 최초 공개, MXNet용 모델 서버로 시작)
- Prerequisites: Java 8, MXNet (단, MXNet 사용 시에만)
- MMS는 프레임워크에 구애받지 않도록 설계되었기 때문에, 모든 프레임워크의 백엔드 엔진 역할을 할 수 있는 충분한 유연성을 제공합니다.
- SageMaker MXNet 추론 컨테이너와 PyTorch 추론 컨테이너는 SageMaker inference toolkit으로 MMS를 래핑하여 사용합니다.
    - 2020년 4월 말 PyTorch용 배포 웹 서비스인 torchserve가 출시되면서, 향후 PyTorch 추론 컨테이너는 MMS 기반에서 torchserve 기반으로 마이그레이션될 예정입니다. 

In [1]:
%%writefile ./src/inference.py

import os
import pandas as pd
import gluonts 
import numpy as np
import argparse
import json
import pathlib
from mxnet import gpu, cpu
from mxnet.context import num_gpus
import matplotlib.pyplot as plt

from gluonts.dataset.util import to_pandas
from gluonts.mx.distribution import DistributionOutput, StudentTOutput, NegativeBinomialOutput, GaussianOutput
from gluonts.model.deepar import DeepAREstimator
from gluonts.mx.trainer import Trainer
from gluonts.evaluation import Evaluator
from gluonts.evaluation.backtest import make_evaluation_predictions, backtest_metrics
from gluonts.model.predictor import Predictor
from gluonts.dataset.field_names import FieldName
from gluonts.dataset.common import ListDataset


def model_fn(model_dir):
    path = pathlib.Path(model_dir)   
    predictor = Predictor.deserialize(path)
    print("model was loaded successfully")
    return predictor


def transform_fn(model, request_body, content_type='application/json', accept_type='application/json'):

    related_cols = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment'] 
    item_cols = ['Type', 'Size'] 
    FREQ = 'W'
    pred_length = 12

    data = json.loads(request_body)    
    
    target_test_df = pd.DataFrame(data['target_values'], index=data['timestamp'])
    related_test_df = pd.DataFrame(data['related_values'], index=data['timestamp'])
    item_df = pd.DataFrame(data['item'], index=data['store_id'])
    item_df.columns = item_cols
        
    target = target_test_df.values
    num_steps, num_series = target_test_df.shape
    start_dt = target_test_df.index[0]
    
    num_related_cols = len(related_cols)
    num_features_per_feature = int(related_test_df.shape[1] / num_related_cols)
    related_list = []

    for feature_idx in range(0, num_related_cols):
        start_idx = feature_idx * num_features_per_feature
        end_idx = start_idx + num_features_per_feature
        related_list.append(related_test_df.iloc[:, start_idx:end_idx].values)

    test_lst = []
    for i in range(0, num_series):
        target_vec = target[:-pred_length, i]
        related_vecs = [related[:, i] for related in related_list]
        item = item_df.loc[i+1]    
        dic = {FieldName.TARGET: target_vec, 
               FieldName.START: start_dt,
               FieldName.FEAT_DYNAMIC_REAL: related_vecs,
               FieldName.FEAT_STATIC_CAT: [item[0]],
               FieldName.FEAT_STATIC_REAL: [item[1]]
              } 
        test_lst.append(dic)

    test_ds = ListDataset(test_lst, freq=FREQ)

    response_body = {}
    forecast_it = model.predict(test_ds)

    for idx, f in enumerate(forecast_it):
        response_body[f'store_{idx}'] = f.samples.mean(axis=0).tolist()

    return json.dumps(response_body)

Overwriting ./src/inference.py


<br>

## 2. Test Inference code 
---

엔드포인트 배포 전, 추론 스크립트를 검증합니다. 

In [2]:
%store -r

In [3]:
from src.inference import model_fn, transform_fn
import json
import numpy as np
import pandas as pd

# Prepare test data 
target_test_df = pd.read_csv("data/target_train.csv", index_col=0, header=[0,1])
related_test_df = pd.read_csv("data/related_train.csv", index_col=0, header=[0,1])
item_df = pd.read_csv("data/item.csv", index_col=0)

input_data = {'target_values': target_test_df.values.tolist(), 
              'related_values': related_test_df.values.tolist(),
              'item': item_df.values.tolist(),
              'timestamp': related_test_df.index.tolist(),
              'store_id': item_df.index.tolist()
             }

request_body = json.dumps(input_data)
# Test inference script 
model = model_fn('./model')
response = transform_fn(model, request_body)
outputs = json.loads(response)
print(outputs['store_0'])

model was loaded successfully
[16.235092163085938, 16.1125545501709, 16.108495712280273, 16.64717674255371, 16.730663299560547, 16.33971405029297, 16.174470901489258, 17.156557083129883, 17.097583770751953, 16.813976287841797, 16.475114822387695, 18.01433563232422]


<br>

## 3. Local Endpoint Inference
---

충분한 검증 및 테스트 없이 훈련된 모델을 곧바로 실제 운영 환경에 배포하기에는 많은 위험 요소들이 있습니다. 따라서, 로컬 모드를 사용하여 실제 운영 환경에 배포하기 위한 추론 인스턴스를 시작하기 전에 노트북 인스턴스의 로컬 환경에서 모델을 배포하는 것을 권장합니다. 이를 로컬 모드 엔드포인트(Local Mode Endpoint)라고 합니다.

In [4]:
import os
import time
import sagemaker
from sagemaker.mxnet import MXNetModel
role = sagemaker.get_execution_role()

In [5]:
local_model_path = f'file://{os.getcwd()}/model/model.tar.gz'
endpoint_name = "local-endpoint-walmart-sale-forecast-{}".format(int(time.time()))


아래 코드 셀을 실행 후, 로그를 확인해 보세요. MMS에 대한 세팅값들을 확인하실 수 있습니다.

```bash
algo-1-u3xwd_1  | MMS Home: /usr/local/lib/python3.6/site-packages
algo-1-u3xwd_1  | Current directory: /
algo-1-u3xwd_1  | Temp directory: /home/model-server/tmp
algo-1-u3xwd_1  | Number of GPUs: 0
algo-1-u3xwd_1  | Number of CPUs: 2
algo-1-u3xwd_1  | Max heap size: 878 M
algo-1-u3xwd_1  | Python executable: /usr/local/bin/python3.6
algo-1-u3xwd_1  | Config file: /etc/sagemaker-mms.properties
algo-1-u3xwd_1  | Inference address: http://0.0.0.0:8080
algo-1-u3xwd_1  | Management address: http://0.0.0.0:8080
algo-1-u3xwd_1  | Model Store: /.sagemaker/mms/models
...
```

In [6]:
local_model = MXNetModel(model_data=local_model_path,
                         role=role,
                         source_dir='src',
                         entry_point='inference.py',
                         framework_version='1.8.0',
                         py_version='py37')

predictor = local_model.deploy(instance_type='local', 
                           initial_instance_count=1, 
                           endpoint_name=endpoint_name,
                           wait=True)

Attaching to 3j551y0yug-algo-1-df9jt
3j551y0yug-algo-1-df9jt | Collecting pandas==1.1.5
3j551y0yug-algo-1-df9jt |   Downloading pandas-1.1.5-cp37-cp37m-manylinux1_x86_64.whl (9.5 MB)
     |████████████████████████████████| 9.5 MB 35.2 MB/s eta 0:00:01
3j551y0yug-algo-1-df9jt | Collecting gluonts==0.7.6
3j551y0yug-algo-1-df9jt |   Downloading gluonts-0.7.6-py3-none-any.whl (897 kB)
     |████████████████████████████████| 897 kB 54.3 MB/s eta 0:00:01
3j551y0yug-algo-1-df9jt | Collecting pydantic~=1.1
3j551y0yug-algo-1-df9jt |   Downloading pydantic-1.8.2-cp37-cp37m-manylinux2014_x86_64.whl (10.1 MB)
     |████████████████████████████████| 10.1 MB 56.4 MB/s eta 0:00:01
3j551y0yug-algo-1-df9jt | Requirement already satisfied: tqdm~=4.23 in /usr/local/lib/python3.7/site-packages (from gluonts==0.7.6->-r /opt/ml/model/code/requirements.txt (line 2)) (4.60.0)
3j551y0yug-algo-1-df9jt | Requirement already satisfied: matplotlib~=3.0 in /usr/local/lib/python3.7/site-packages (from gluonts==0.7.6

3j551y0yug-algo-1-df9jt | 2021-06-30 14:12:02,198 [INFO ] main com.amazonaws.ml.mms.ModelServer - Inference API bind to: http://0.0.0.0:8080
3j551y0yug-algo-1-df9jt | Model server started.
3j551y0yug-algo-1-df9jt | 2021-06-30 14:12:02,210 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Connection accepted: /home/model-server/tmp/.mms.sock.9000.
3j551y0yug-algo-1-df9jt | 2021-06-30 14:12:02,210 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Connection accepted: /home/model-server/tmp/.mms.sock.9000.
3j551y0yug-algo-1-df9jt | 2021-06-30 14:12:02,211 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Connection accepted: /home/model-server/tmp/.mms.sock.9000.
3j551y0yug-algo-1-df9jt | 2021-06-30 14:12:02,212 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Connection accepted: /home/model-server/tmp/.mms.sock.9000.
3j551y0yug-algo-1-df9jt | 2021-06-30 14:12:02,218 [WARN ] pool-2-thread-1 com.amazonaws

로컬에서 컨테이너를 배포했기 때문에 컨테이너가 현재 실행 중임을 확인할 수 있습니다.

In [7]:
!docker ps

CONTAINER ID        IMAGE                                                                         COMMAND                  CREATED             STATUS              PORTS                              NAMES
797dbd73418e        763104351884.dkr.ecr.us-east-1.amazonaws.com/mxnet-inference:1.8.0-cpu-py37   "python /usr/local/b…"   24 seconds ago      Up 20 seconds       0.0.0.0:8080->8080/tcp, 8081/tcp   3j551y0yug-algo-1-df9jt


### Inference using SageMaker SDK

SageMaker SDK의 `predict()` 메서드로 쉽게 추론을 수행할 수 있습니다. 

In [8]:
outputs = predictor.predict(input_data)
print(outputs['store_0'], outputs['store_20'])

3j551y0yug-algo-1-df9jt | 2021-06-30 14:12:18,213 [INFO ] W-9000-model com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time: 986
3j551y0yug-algo-1-df9jt | 2021-06-30 14:12:18,214 [INFO ] W-9000-model ACCESS_LOG - /172.19.0.1:53626 "POST /invocations HTTP/1.1" 200 991
[16.23509407043457, 16.1125545501709, 16.108495712280273, 16.64717674255371, 16.730663299560547, 16.33971405029297, 16.174468994140625, 17.156557083129883, 17.097583770751953, 16.81397819519043, 16.475114822387695, 18.01433563232422] [7.053330898284912, 7.185001373291016, 7.335565090179443, 7.1755452156066895, 7.255197048187256, 7.280794620513916, 7.130575656890869, 7.46973991394043, 7.510270595550537, 7.223666191101074, 7.417709827423096, 7.8838300704956055]


### Inference using Boto3 SDK

SageMaker SDK의 `predict()` 메서드로 추론을 수행할 수도 있지만, 이번에는 boto3의 `invoke_endpoint()` 메서드로 추론을 수행해 보겠습니다.<br>
Boto3는 서비스 레벨의 low-level SDK로, ML 실험에 초점을 맞춰 일부 기능들이 추상화된 high-level SDK인 SageMaker SDK와 달리
SageMaker API를 완벽하게 제어할 수 있습으며, 프로덕션 및 자동화 작업에 적합합니다.

참고로 `invoke_endpoint()` 호출을 위한 런타임 클라이언트 인스턴스 생성 시, 로컬 배포 모드에서는 `sagemaker.local.LocalSagemakerRuntimeClient()`를 호출해야 합니다.

In [9]:
client = sagemaker.local.LocalSagemakerClient()
runtime_client = sagemaker.local.LocalSagemakerRuntimeClient()
endpoint_name = local_model.endpoint_name

response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name, 
    ContentType='application/json',
    Accept='application/json',
    Body=json.dumps(input_data)
    )
outputs = json.loads(response['Body'].read().decode())

3j551y0yug-algo-1-df9jt | 2021-06-30 14:12:24,867 [INFO ] W-9000-model com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time: 972
3j551y0yug-algo-1-df9jt | 2021-06-30 14:12:24,867 [INFO ] W-9000-model ACCESS_LOG - /172.19.0.1:53630 "POST /invocations HTTP/1.1" 200 974


In [10]:
print(outputs['store_0'], outputs['store_20'])

[16.23509407043457, 16.1125545501709, 16.108495712280273, 16.64717674255371, 16.730663299560547, 16.33971405029297, 16.174468994140625, 17.156557083129883, 17.097583770751953, 16.81397819519043, 16.475114822387695, 18.01433563232422] [7.053330898284912, 7.185001373291016, 7.335565090179443, 7.1755452156066895, 7.255197048187256, 7.280794620513916, 7.130575656890869, 7.46973991394043, 7.510270595550537, 7.223666191101074, 7.417709827423096, 7.8838300704956055]


### Local Mode Endpoint Clean-up

엔드포인트를 계속 사용하지 않는다면, 엔드포인트를 삭제해야 합니다. 
SageMaker SDK에서는 `delete_endpoint()` 메소드로 간단히 삭제할 수 있습니다.

In [11]:
def delete_endpoint(client, endpoint_name):
    response = client.describe_endpoint_config(EndpointConfigName=endpoint_name)
    model_name = response['ProductionVariants'][0]['ModelName']

    client.delete_model(ModelName=model_name)    
    client.delete_endpoint(EndpointName=endpoint_name)
    client.delete_endpoint_config(EndpointConfigName=endpoint_name)    
    
    print(f'--- Deleted model: {model_name}')
    print(f'--- Deleted endpoint: {endpoint_name}')
    print(f'--- Deleted endpoint_config: {endpoint_name}')    
delete_endpoint(client, endpoint_name)

Gracefully stopping... (press Ctrl+C again to force)
--- Deleted model: mxnet-inference-2021-06-30-14-11-19-730
--- Deleted endpoint: local-endpoint-walmart-sale-forecast-1625062271
--- Deleted endpoint_config: local-endpoint-walmart-sale-forecast-1625062271


<br>

## 4. SageMaker Hosted Endpoint Inference
---

이제 실제 운영 환경에 엔드포인트 배포를 수행해 보겠습니다. 로컬 모드 엔드포인트와 대부분의 코드가 동일하며, 모델 아티팩트 경로(`model_data`)와 인스턴스 유형(`instance_type`)만 변경해 주시면 됩니다. SageMaker가 관리하는 배포 클러스터를 프로비저닝하는 시간이 소요되기 때문에 추론 서비스를 시작하는 데에는 약 5~10분 정도 소요됩니다.

In [12]:
import os
import boto3
import sagemaker
from sagemaker.mxnet import MXNet

boto_session = boto3.Session()
sagemaker_session = sagemaker.Session(boto_session=boto_session)
role = sagemaker.get_execution_role()
bucket = sagemaker.Session().default_bucket()

In [13]:
model_path = os.path.join(s3_model_dir, "model.tar.gz")
endpoint_name = "endpoint-walmart-sale-forecast-{}".format(int(time.time()))

In [14]:
model = MXNetModel(model_data=model_path,
                         role=role,
                         source_dir='src',
                         entry_point='inference.py',
                         framework_version='1.8.0',
                         py_version='py37')

predictor = model.deploy(instance_type="ml.c5.large", 
                         initial_instance_count=1, 
                         endpoint_name=endpoint_name,
                         wait=True)

-------------!

추론을 수행합니다. 로컬 모드의 코드와 동일합니다.

In [15]:
import boto3
client = boto3.client('sagemaker')
runtime_client = boto3.client('sagemaker-runtime')
endpoint_name = model.endpoint_name

In [16]:
response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name, 
    ContentType='application/json',
    Accept='application/json',
    Body=json.dumps(input_data)
    )
outputs = json.loads(response['Body'].read().decode())

In [17]:
print(outputs['store_0'], outputs['store_20'])

[16.23509407043457, 16.1125545501709, 16.108495712280273, 16.64717674255371, 16.730663299560547, 16.33971405029297, 16.174468994140625, 17.156557083129883, 17.097583770751953, 16.81397819519043, 16.475114822387695, 18.01433563232422] [7.053330898284912, 7.185001373291016, 7.335565090179443, 7.1755452156066895, 7.255197048187256, 7.280794620513916, 7.130575656890869, 7.46973991394043, 7.510270595550537, 7.223666191101074, 7.417709827423096, 7.8838300704956055]


### SageMaker Hosted Endpoint Clean-up

엔드포인트를 계속 사용하지 않는다면, 불필요한 과금을 피하기 위해 엔드포인트를 삭제해야 합니다. 
SageMaker SDK에서는 `delete_endpoint()` 메소드로 간단히 삭제할 수 있으며, UI에서도 쉽게 삭제할 수 있습니다.

In [18]:
delete_endpoint(client, endpoint_name)

--- Deleted model: mxnet-inference-2021-06-30-14-12-57-286
--- Deleted endpoint: endpoint-walmart-sale-forecast-1625062360
--- Deleted endpoint_config: endpoint-walmart-sale-forecast-1625062360
